In [2]:
import os
import glob
from dotenv import load_dotenv

In [3]:
# imports for langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
# from langchain_chroma import Chroma
from langchain.vectorstores import FAISS
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [4]:
# imports for langchain, plotly and Chroma

from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_community.vectorstores import Chroma

In [5]:
#MODEL = Ollama(model="llama3.2")
#MODEL = "gpt-4o-mini"
db_name = "vector-database"

In [6]:
folders = glob.glob("example/*")

In [7]:
text_loader_kwargs = {'encoding': 'utf-8'}

In [16]:
import jsonlines
import json
with jsonlines.open("articles.json","r") as documents:
    print(next(iter(documents)))

{'url': 'https://arxiv.org/pdf/2405.10825', 'title': 'Large Language Model (LLM) for Telecommunications: A Comprehensive Survey on Principles, Key Techniques, and Opportunities', 'time': '2023-10-17T14:00:32.000Z', 'tags': ['Large Language Model', 'Telecom', 'Article', '5G'], 'authors': ['Hao Zhou', 'Chengming Hu', 'Ye Yuan', 'Yufei Cui', 'Yili Jin']}


In [24]:
from langchain.document_loaders import PyPDFLoader
from langchain_core.documents import Document
folders = glob.glob("example/*")

In [99]:
langchain_documents = []

with jsonlines.open("articles.json", "r") as metadata_file:
    for file_path, metadata in zip(folders, metadata_file):
        try:
            pdf_loader = PyPDFLoader(file_path)
            pdf_documents = pdf_loader.load()

            for pdf_doc in pdf_documents:
            # Metadata'yı dökümana ekle
                pdf_doc.metadata.update({
                    "url": json.dumps(metadata.get("url")),
                    "tags": json.dumps(metadata.get("tags")),
                    "title": metadata.get("title"),
                    "authors": json.dumps(metadata.get("authors"))
                })

            # Dökümanı listeye ekle
                langchain_documents.append( Document(
                        page_content=pdf_doc.page_content,
                        metadata=pdf_doc.metadata
                    )
                )
        except Exception as e:
            print(f"Error loading file {file_path}: {e}")

In [100]:
'''
!pip install pdfminer.six
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar,LTLine,LAParams
import os

path=r'example/Rapor-RAGEvaluation.pdf'

Extract_Data=[]

for page_layout in extract_pages(path):
    for element in page_layout:
        if isinstance(element, LTTextContainer):
            for text_line in element:
                for character in text_line:
                    if isinstance(character, LTChar):
                        Font_size=character.size
            Extract_Data.append([Font_size,(element.get_text())])
'''

"\n!pip install pdfminer.six\nfrom pdfminer.high_level import extract_pages\nfrom pdfminer.layout import LTTextContainer, LTChar,LTLine,LAParams\nimport os\n\npath=r'example/Rapor-RAGEvaluation.pdf'\n\nExtract_Data=[]\n\nfor page_layout in extract_pages(path):\n    for element in page_layout:\n        if isinstance(element, LTTextContainer):\n            for text_line in element:\n                for character in text_line:\n                    if isinstance(character, LTChar):\n                        Font_size=character.size\n            Extract_Data.append([Font_size,(element.get_text())])\n"

In [101]:
'''font_dict = {}
for element in Extract_Data:
    font_dict.update({element[0] : element[1]})
list_of_fonts = list(font_dict.keys())
list_of_fonts.sort(reverse=True)
list_of_fonts
print(f"Title is {font_dict[list_of_fonts[0]]}")
'''

'font_dict = {}\nfor element in Extract_Data:\n    font_dict.update({element[0] : element[1]})\nlist_of_fonts = list(font_dict.keys())\nlist_of_fonts.sort(reverse=True)\nlist_of_fonts\nprint(f"Title is {font_dict[list_of_fonts[0]]}")\n'

In [102]:
font_dict

{28.0: 'REFERENCES\n',
 18.0: 'TruLens\n',
 12.0: '6- https://arxiv.org/abs/2311.09476\n',
 13.5: 'Context recall : 1/2 = 0.5\n',
 13.500000000000028: 'level. It is a vital substance for all known forms of life.”]\n',
 16.0: 'ILLUSTRATION\n',
 13.499999999999943: 'To compare the N generated question, we use cosine or dot product vector similarity \n',
 11.999999999999943: 'EXAMPLE\n',
 16.00000000000003: 'RAG METRICS\n'}

In [103]:
len(langchain_documents), langchain_documents

(11,
 [Document(metadata={'source': 'example/Rapor-RAGEvaluation.pdf', 'page': 0, 'url': '"https://arxiv.org/pdf/2405.10825"', 'tags': '["Large Language Model", "Telecom", "Article", "5G"]', 'title': 'Large Language Model (LLM) for Telecommunications: A Comprehensive Survey on Principles, Key Techniques, and Opportunities', 'authors': '["Hao Zhou", "Chengming Hu", "Ye Yuan", "Yufei Cui", "Yili Jin"]'}, page_content="Evaluation Methods\nGroundness(Faithfulness)\nGroundness uses OpenAI LLMs or HuggingFace NLI to attempt to check if an answer is grounded \nin its supplied contexts on a scale from 1 to 10. You can monitor your RAG pipelines' groundedness \nscore with evaluation tools. A grounded response adheres closely to the given information, avoiding \nspeculation or fabrication. In groundedness measurements, source information is crucial and serves \nas the grounding source.To calculate it, we start by extracting all claims from the LLM provided \nanswer first. Then for each claim we 

In [104]:
#from langchain.vectorstores import Qdrant
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

In [105]:
from langchain_ollama import OllamaEmbeddings

# Load environment variables in a file called .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
embeddings = OpenAIEmbeddings()

In [106]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(langchain_documents)

In [107]:
"""import re
import nltk

def document_based_chunking(text):
    paragraphs = re.split(r'\n\s*\n', text)
    chunks = []
    
    for paragraph in paragraphs:
        sentences = nltk.sent_tokenize(paragraph)
        current_chunk = ""
        
        for sentence in sentences:
            if len(current_chunk) + len(sentence) <= 100:
                current_chunk += sentence + " "
            else:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + " "
        
        if current_chunk:
            chunks.append(current_chunk.strip())
    
    return chunks

# Store chunks for each document
all_chunks = []

for document in langchain_documents:
    text = document.page_content  # This might not be the correct way to access the content
    if hasattr(document, 'text'):
        text = document.text  # Ensure you're using the correct attribute to access the text
    
    chunks = document_based_chunking(text)
    all_chunks.extend(chunks)  # Add the chunks to the all_chunks list

# Now all_chunks contains all the chunks from all documents
"""

'import re\nimport nltk\n\ndef document_based_chunking(text):\n    paragraphs = re.split(r\'\n\\s*\n\', text)\n    chunks = []\n    \n    for paragraph in paragraphs:\n        sentences = nltk.sent_tokenize(paragraph)\n        current_chunk = ""\n        \n        for sentence in sentences:\n            if len(current_chunk) + len(sentence) <= 100:\n                current_chunk += sentence + " "\n            else:\n                chunks.append(current_chunk.strip())\n                current_chunk = sentence + " "\n        \n        if current_chunk:\n            chunks.append(current_chunk.strip())\n    \n    return chunks\n\n# Store chunks for each document\nall_chunks = []\n\nfor document in langchain_documents:\n    text = document.page_content  # This might not be the correct way to access the content\n    if hasattr(document, \'text\'):\n        text = document.text  # Ensure you\'re using the correct attribute to access the text\n    \n    chunks = document_based_chunking(text)

In [109]:
    

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
#vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
# create a new Chat with OpenAI
#llm = ChatOpenAI(temperature=0.7, model_name=MODEL)

from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
) 

# set up the conversation memory for the chat
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# the retriever is an abstraction over the VectorStore that will be used during RAG
retriever = vectorstore.as_retriever()

# putting it together: set up the conversation chain with the GPT 3.5 LLM, the vector store and memory
#conversation_chain = ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

query = "What is groundedness score"
result = conversation_chain.invoke({"question":query})
print(result["answer"])


'''
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg
'''

The follow-up input can be rephrased as a standalone question:

"What does a higher groundedness score indicate about an answer's relationship to its context?"

InvalidCollectionException: Collection 507fdc44-581a-4f53-86a5-d0de1840f553 does not exist.

In [46]:
store = Qdrant.from_documents(
    langchain_documents,
    embeddings,
    path="/tmp/ai_qdrant",
    collection_name="AI-Embeddings",
)  

In [34]:
vectorstore.similarity_search_with_score(
    query="AI and authors",
    k=1
)

[(Document(metadata={'authors': '["Hao Zhou", "Chengming Hu", "Ye Yuan", "Yufei Cui", "Yili Jin"]', 'page': 4, 'source': 'example/Rapor-RAGEvaluation.pdf', 'tags': '["Large Language Model", "Telecom", "Article", "5G"]', 'title': 'Large Language Model (LLM) for Telecommunications: A Comprehensive Survey on Principles, Key Techniques, and Opportunities', 'url': '"https://arxiv.org/pdf/2405.10825"'}, page_content='ILLUSTRATION'),
  0.4380386306239418)]

In [38]:
vectorstore.similarity_search_with_score(
    query="AI and authors",
    filter={"title": 'Large Language Model (LLM) for Telecommunications: A Comprehensive Survey on Principles, Key Techniques, and Opportunities' },
    k=1
)  

[(Document(metadata={'authors': '["Hao Zhou", "Chengming Hu", "Ye Yuan", "Yufei Cui", "Yili Jin"]', 'page': 4, 'source': 'example/Rapor-RAGEvaluation.pdf', 'tags': '["Large Language Model", "Telecom", "Article", "5G"]', 'title': 'Large Language Model (LLM) for Telecommunications: A Comprehensive Survey on Principles, Key Techniques, and Opportunities', 'url': '"https://arxiv.org/pdf/2405.10825"'}, page_content='ILLUSTRATION'),
  0.4380386306239418)]

In [1]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(
    model="llama3.2",
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
) 

/tmp/ipykernel_83196/2562771045.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(
/tmp/ipykernel_83196/2562771045.py:5: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(


In [50]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.prompts import PromptTemplate

In [51]:
def retrieval_chain_with_filter(llm, filter={}):
    template = """You are a bot that answers user questions using only the context provided.
    If you don't know the answer, simply state that you don't know.
    {context}
    Question: {input}"""

    prompt = PromptTemplate(template=template, input_variables=["context", "input"])
    retriever = store.as_retriever(search_kwargs={'filter': filter})
    llm_with_prompt = create_stuff_documents_chain(llm, prompt)
    return create_retrieval_chain(retriever, llm_with_prompt)  

In [53]:
result = retrieval_chain_with_filter(llm).invoke({
    "input": ""
})

ValueError: Ollama call failed with status code 500. Details: {"error":"model requires more system memory (3.4 GiB) than is available (2.9 GiB)"}

In [54]:
!pip uninstall fastembed

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Found existing installation: fastembed 0.4.2
Uninstalling fastembed-0.4.2:
  Would remove:
    /home/yasar/anaconda3/envs/llms/lib/python3.11/site-packages/fastembed-0.4.2.dist-info/*
    /home/yasar/anaconda3/envs/llms/lib/python3.11/site-packages/fastembed/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [55]:
y

NameError: name 'y' is not defined

In [1]:
%history

%history


In [2]:
%history

%history
%history


In [3]:
%history -g -f Untitled1.ipynb